In [1]:
class Order:
    def __init__(self, symbol: str, qty: int):
        self.symbol = symbol
        self.qty = qty

class Trade:
    def __init__(self, symbol: str, qty: int, price: float):
        self.symbol = symbol
        self.qty = qty
        self.price = price

class Position:
    def __init__(self, symbol: str):
        self.symbol = symbol
        self.qty = 0

    def add(self, qty: int):
        self.qty += qty  # `self` is the instance being mutated

class Quote:
    def __init__(self, bids: list[float], asks: list[float]):
        self.bids = bids
        self.asks = asks

    def __repr__(self):  # helpful for debugging
        return f"Quote(bid={max(self.bids, default=None)}, ask={min(self.asks, default=None)})"

    def __len__(self):  # treat as sized container of price levels
        return len(self.bids) + len(self.asks)

from dataclasses import dataclass

@dataclass(frozen=True) # immutable
class Ticker:
    symbol: str
    exchange: str

@dataclass
class OrderRequest:
    symbol: str
    qty: int
    side: str  # "BUY" or "SELL"

from abc import ABC, abstractmethod

class FeeModel(ABC):
    @abstractmethod
    def fee(self, notional: float) -> float: ...

class FixedBpsFee(FeeModel):
    def __init__(self, bps: float):
        self.bps = bps
    def fee(self, notional: float) -> float:
        return notional * self.bps / 10_000

class MinFloorFee(FeeModel):
    def __init__(self, bps: float, minimum: float):
        self.bps = bps
        self.minimum = minimum
    def fee(self, notional: float) -> float:
        return max(notional * self.bps / 10_000, self.minimum)

class Broker:
    def __init__(self, fee_model: FeeModel):
        self.fee_model = fee_model
    def commission(self, notional: float) -> float:
        return self.fee_model.fee(notional)


class BrokerageAccount:
    def __init__(self):
        self._balance = 0.0  # protected by convention, _ attributes are considered "private"

    @property
    def balance(self) -> float:
        return self._balance

    def deposit(self, amount: float) -> None:
        if amount <= 0:
            raise ValueError("deposit must be positive")
        self._balance += amount

    def withdraw(self, amount: float) -> None:
        if amount <= 0:
            raise ValueError("withdraw must be positive")
        if amount > self._balance:
            raise ValueError("insufficient funds")
        self._balance -= amount

In [2]:
symbol = "AAPL"
qty = 100
price = 268.84
bid = -0.1
ask = +0.1
exchange = "NASDAQ"
order_type = "BUY"
order = Order(symbol, qty)
trade = Trade(symbol, qty, price)
quote = Quote( price + bid, price + ask)
ticker = Ticker(symbol,exchange)
order_request = OrderRequest(symbol, qty, order_type)

b1 = Broker(FixedBpsFee(1.2))
b2 = Broker(MinFloorFee(0.8, 2.0))

my_brokerage_account= BrokerageAccount()
my_brokerage_account.deposit(13425436)

In [3]:
my_brokerage_account.balance

13425436.0

In [4]:
from dataclasses import dataclass

@dataclass
class Fill: 
    """ A filled order/trade """
    symbol: str
    qty: int
    price: float

class ExecutionEngine1:
    """ 
    Simple execution engine that processes orders through a broker and updates an account.
    """
    def __init__(self, broker: Broker, account: BrokerageAccount):
        self.broker = broker
        self.account = account

    def execute(self, req: OrderRequest, price: float) -> Fill:
        notional = abs(req.qty) * price
        (self.broker)
        fee = self.broker.commission(notional)
        cash_change = -notional - fee if req.side == "BUY" else notional - fee
        if self.account.balance + cash_change < 0:
            raise ValueError("order would make balance negative")
        if cash_change > 0:
            self.account.deposit(cash_change)
        else:
            self.account.withdraw(-cash_change)
        return Fill(req.symbol, req.qty, price)

In [5]:
exec_engine = ExecutionEngine1(broker = b2, account = my_brokerage_account)
exec_engine.execute(req = order_request, price= 1000)


Fill(symbol='AAPL', qty=100, price=1000)

In [6]:
my_brokerage_account.balance

13325428.0

In [7]:
from market_data_loader.data_handler import *

In [8]:
tickers = ["AAPL", "SPY", "TLT"]
handler = OnlineDataHandler(tickers)
handler.load_data()

print("\nDernier bar pour AAPL :")
print(handler.get_latest_bar("AAPL"))

print("\nVariables macroéconomiques :")
print(handler.macro_data.tail())

# Exemple : données locales
csv_handler = HistoricCSVDataHandler("data/", ["AAPL"])
print("\nDernier bar pour AAPL_D :")
print(csv_handler.get_latest_bar("AAPL"))

Chargement des données de marché depuis Alpaca...
Market data loaded:
Frequency: 1Day
Period: 2023-01-04 → 2025-11-04
712 observations, 3 tickers
Chargement des données macroéconomiques depuis FRED...
FRED data loaded:
Period: 2023-01-01 → 2025-10-31
1035 observations, 7 variables
Macro variables built:
Period: 2023-01-01 → 2025-10-31
1035 observations, 8 variables
✓ Données chargées avec succès.

Dernier bar pour AAPL :
close             268.780000
high              269.590000
low               267.670000
trade_count      5025.000000
open              268.290000
volume         364375.000000
vwap              268.622748
Name: 2025-11-04 05:00:00+00:00, dtype: float64

Variables macroéconomiques :
                CPI   INDPPI  M1SUPPLY     CCREDIT  BMINUSA  AAA10Y  TB3MS  \
DATE                                                                         
2025-10-27  324.368  262.443   18912.8  5061167.32     0.60    1.03   3.82   
2025-10-28  324.368  262.443   18912.8  5061167.32     0.60 

In [44]:
from abc import ABC, abstractmethod
from typing import Dict, List
import pandas as pd
from dataclasses import dataclass
from enum import Enum # Enum is a module that provides support for enumerations

class SignalType(Enum):
    """ Enumeration for signal types """
    BUY = "BUY"
    SELL = "SELL"
    HOLD = "HOLD"
    EXIT = "EXIT"

@dataclass
class Signal:
    """ Represents a trading signal """
    symbol: str
    signal_type: SignalType
    price: float
    volume: float

    def __post_init__(self):
    #    if self.volume <= 0:
     #       raise ValueError("Volume must be positive")

        if self.signal_type not in SignalType:
            raise ValueError(f"Invalid signal type: {self.signal_type}")

class Strategy(ABC):
    """ Abstract base class for trading strategies """
    @abstractmethod
    def generate_signals(self, data: Dict[str, pd.DataFrame]) -> List[Signal]:
        """ Generates trading signals based on the provided data """
        pass

class MovingAverageCrossoverStrategy(Strategy):
    """ Simple moving average crossover strategy """
    def __init__(self, short_window: int = 40, long_window: int = 100):
        self.short_window = short_window
        self.long_window = long_window

    def generate_signals(self, data: Dict[str, pd.DataFrame]) -> List[Signal]:
        signals = []
        for symbol, df in data.items():
            df['short_mavg'] = df['close'].rolling(window=self.short_window, min_periods=1).mean()
            df['long_mavg'] = df['close'].rolling(window=self.long_window, min_periods=1).mean()

            if df['short_mavg'].iloc[-1] > df['long_mavg'].iloc[-1]:
                signals.append(Signal(symbol=symbol, signal_type=SignalType.BUY, price=df['close'].iloc[-1], volume=100))
            elif df['short_mavg'].iloc[-1] < df['long_mavg'].iloc[-1]:
                signals.append(Signal(symbol=symbol, signal_type=SignalType.SELL, price=df['close'].iloc[-1], volume=100))
            else:
                signals.append(Signal(symbol=symbol, signal_type=SignalType.HOLD, price=df['close'].iloc[-1], volume=0))
        return signals

In [19]:
strategy = MovingAverageCrossoverStrategy(short_window=2, long_window=3)

Portfolio

In [22]:
from dataclasses import dataclass, field
from typing import Dict

@dataclass
class Portfolio:
    cash: float = 0.0
    positions: Dict[str, int] = field(default_factory=dict)

    def qty(self, symbol: str) -> int:
        return self.positions.get(symbol, 0)

    def apply_fill(self, fill: Fill, fee: float) -> None:
        sign = 1 if fill.qty > 0 else -1
        notional = abs(fill.qty) * fill.price
        # BUY reduces cash, SELL increases cash
        self.cash += (-notional - fee) if sign > 0 else (notional - fee)
        self.positions[fill.symbol] = self.qty(fill.symbol) + fill.qty
        if self.positions[fill.symbol] == 0:
            self.positions.pop(fill.symbol)

    def market_value(self, prices: Dict[str, float]) -> float:
        return sum(self.qty(sym) * prices.get(sym, 0.0) for sym in self.positions)

    def equity(self, prices: Dict[str, float]) -> float:
        return self.cash + self.market_value(prices)

In [23]:
portfolio = Portfolio(cash=100_000)

BROKER

In [24]:
from typing import Tuple

class BacktestBroker:
    def __init__(self, fee_model: FeeModel, slippage_bps: float = 0.0):
        self.fee_model = fee_model
        self.slippage_bps = slippage_bps

    def _apply_slippage(self, price: float, side: SignalType) -> float:
        # BUY pays up, SELL gets less
        adj = price * (self.slippage_bps / 10_000.0)
        return price + adj if side == SignalType.BUY else price - adj

    def order_for_signal(self, sig: Signal) -> OrderRequest:
        if sig.signal_type == SignalType.HOLD or sig.volume == 0:
            return OrderRequest(symbol=sig.symbol, qty=0, side="HOLD")
        side = "BUY" if sig.signal_type == SignalType.BUY else "SELL"
        qty = int(sig.volume) if side == "BUY" else -int(sig.volume)
        return OrderRequest(symbol=sig.symbol, qty=qty, side=side)

    def execute(self, req: OrderRequest, mkt_price: float) -> Tuple[Fill, float]:
        if req.qty == 0:
            return Fill(req.symbol, 0, mkt_price), 0.0
        traded_price = self._apply_slippage(mkt_price, SignalType.BUY if req.qty > 0 else SignalType.SELL)
        notional = abs(req.qty) * traded_price
        fee = self.fee_model.fee(notional)
        return Fill(req.symbol, req.qty, traded_price), fee

In [25]:
import math
from typing import List, Dict

class PerformanceMetrics:
    def __init__(self, equity_curve: List[float]):
        self.equity = equity_curve
        self.returns = [0.0] + [
            (self.equity[i] / self.equity[i-1] - 1.0) for i in range(1, len(self.equity))
        ]

    def summary(self, periods_per_year: int = 252) -> Dict[str, float]:
        if not self.equity or len(self.equity) < 2:
            return {"total_return": 0.0, "vol": 0.0, "sharpe": 0.0, "max_dd": 0.0}
        total_return = self.equity[-1] / self.equity[0] - 1.0
        mean = sum(self.returns[1:]) / max(1, len(self.returns) - 1)
        var = sum((r - mean) ** 2 for r in self.returns[1:]) / max(1, len(self.returns) - 2)
        vol = math.sqrt(var) * math.sqrt(periods_per_year)
        sharpe = (mean * periods_per_year) / vol if vol > 0 else 0.0
        # max drawdown
        peak = self.equity[0]
        max_dd = 0.0
        for v in self.equity:
            peak = max(peak, v)
            dd = (v / peak) - 1.0
            max_dd = min(max_dd, dd)
        return {
            "total_return": total_return,
            "vol": vol,
            "sharpe": sharpe,
            "max_dd": max_dd,
        }

In [31]:
from typing import Dict, List

class Backtester:
    def __init__(self, data: Dict[str, pd.DataFrame], strategy: Strategy, broker: BacktestBroker, portfolio: Portfolio):
        self.data = data
        self.strategy = strategy
        self.broker = broker
        self.portfolio = portfolio
        self.equity_curve: List[float] = []

    def _prices_snapshot(self, t: int) -> Dict[str, float]:
        return {sym: df['close'].iloc[t] for sym, df in self.data.items() if t < len(df)}

    def run(self) -> PerformanceMetrics:
        # assume all symbols share the same index length for brevity
        T = min(len(df) for df in self.data.values())
        for t in range(T):
            # 1) Generate signals using history up to t
            hist = {sym: df.iloc[: t + 1] for sym, df in self.data.items()}
            signals = self.strategy.generate_signals(hist)
            prices = self._prices_snapshot(t)

            # 2) Convert to order requests and execute
            for sig in signals:
                req = self.broker.order_for_signal(sig)
                if req.qty == 0:
                    continue
                mkt_px = prices.get(req.symbol)
                if mkt_px is None:
                    continue
                fill, fee = self.broker.execute(req, mkt_px)
                self.portfolio.apply_fill(fill, fee)

            # 3) Record equity
            self.equity_curve.append(self.portfolio.equity(prices))

        return PerformanceMetrics(self.equity_curve)

In [46]:
data

{'AAPL':    close
 0    100
 1    101
 2    102
 3    101
 4    103,
 'MSFT':    close
 0    100
 1    101
 2    102
 3    101
 4    103}

In [56]:
handler.price_close_df['AAPL'].dtype

pandas.core.series.Series

In [52]:
data

,AAPL,SPY,TLT
DATE,,,
2023-01-03 05:00:00+00:00,125.145,380.920,101.460
2023-01-04 05:00:00+00:00,126.390,383.750,102.870
2023-01-05 05:00:00+00:00,125.030,379.340,103.270
2023-01-06 05:00:00+00:00,129.560,388.010,105.180
2023-01-09 05:00:00+00:00,130.180,387.900,105.715
...,...,...,...
2025-10-29 04:00:00+00:00,269.820,687.315,91.070
2025-10-30 04:00:00+00:00,271.320,680.100,90.560
2025-10-31 04:00:00+00:00,270.410,682.030,90.295


In [58]:
#Inputs
symbols = ["AAPL", "MSFT"]
data = {s: pd.DataFrame({"close": handler.price_close_df[s] }) for s in handler.price_close_df.columns}

strategy = MovingAverageCrossoverStrategy(short_window=2, long_window=3)
broker = BacktestBroker(FixedBpsFee(1.0), slippage_bps=2.0)
portfolio = Portfolio(cash=100_00000000)

bt = Backtester(data, strategy, broker, portfolio)
metrics = bt.run().summary()

print({k: round(v, 4) for k, v in metrics.items()})

/var/folders/3l/s4t1bsn973dfhyz_6q842jr00000gn/T/ipykernel_90402/3697943152.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['short_mavg'] = df['close'].rolling(window=self.short_window, min_periods=1).mean()
/var/folders/3l/s4t1bsn973dfhyz_6q842jr00000gn/T/ipykernel_90402/3697943152.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['long_mavg'] = df['close'].rolling(window=self.long_window, min_periods=1).mean()
/var/folders/3l/s4t1bsn973dfhyz_6q842jr00000gn/T/ipykernel_90402/3697943152.py:45:

{'total_return': 0.0002, 'vol': 0.0001, 'sharpe': 0.87, 'max_dd': -0.0002}


/var/folders/3l/s4t1bsn973dfhyz_6q842jr00000gn/T/ipykernel_90402/3697943152.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['short_mavg'] = df['close'].rolling(window=self.short_window, min_periods=1).mean()
/var/folders/3l/s4t1bsn973dfhyz_6q842jr00000gn/T/ipykernel_90402/3697943152.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['long_mavg'] = df['close'].rolling(window=self.long_window, min_periods=1).mean()
/var/folders/3l/s4t1bsn973dfhyz_6q842jr00000gn/T/ipykernel_90402/3697943152.py:45:

In [36]:
data = handler 
strategy = MovingAverageCrossoverStrategy(short_window=2, long_window=3)
broker = BacktestBroker(FixedBpsFee(1.0), slippage_bps=2.0)
portfolio = Portfolio(cash=100_0000000)

bt = Backtester(data, strategy, broker, portfolio)
metrics = bt.run().summary()

print({k: round(v, 4) for k, v in metrics.items()})

AttributeError: 'OnlineDataHandler' object has no attribute 'values'